In [1]:
import joblib

# Load the logistic regression model
logistic_regression_model = joblib.load('logistic_regression_model.pkl')

# Load the StandardScaler
sc = joblib.load('standard_scaler.pkl')


In [2]:
import os
import librosa
import numpy as np
from sklearn.linear_model import LogisticRegression

def extract_vocal_segment(audio_file, duration=60, threshold=0.01):
    y, sr = librosa.load(audio_file, sr=22050)
    vocal_start_time = 30
    # Calculate start frame and end frame based on vocal_start_time and duration
    start_frame = int(librosa.time_to_samples(vocal_start_time, sr=sr))
    end_frame = start_frame + int(sr * duration)

    # Ensure the end frame is within the audio length
    if end_frame > len(y):
        end_frame = len(y)

    # Extract the vocal segment
    vocal_segment = y[start_frame:end_frame]

    return vocal_segment, sr

def extract_features(audio_path, num_mfcc=20, sample_rate=22050, n_fft=2048, chroma_hop_length=512):
    # Initialize the feature dictionary
    features = {
        "filename": os.path.basename(audio_path),
        "chroma_stft_mean": 0, "chroma_stft_var": 0,
        "rms_mean": 0, "rms_var": 0,
        "spectral_centroid_mean": 0, "spectral_centroid_var": 0,
        "spectral_bandwidth_mean": 0, "spectral_bandwidth_var": 0,
        "rolloff_mean": 0, "rolloff_var": 0,
        "zero_crossing_rate_mean": 0, "zero_crossing_rate_var": 0,
        "harmony_mean": 0, "harmony_var": 0,
        "perceptr_mean": 0, "perceptr_var": 0,
        "tempo": 0
    }

    for i in range(1, num_mfcc+1):
        features[f"mfcc{i}_mean"] = 0
        features[f"mfcc{i}_var"] = 0

    # Extract the vocal segment
    vocal_segment, sr = extract_vocal_segment(audio_path)

    # Chromagram
    chromagram = librosa.feature.chroma_stft(y=vocal_segment, sr=sr, hop_length=chroma_hop_length)
    features["chroma_stft_mean"] = chromagram.mean()
    features["chroma_stft_var"] = chromagram.var()
    
    # Root Mean Square Energy
    RMSEn = librosa.feature.rms(y=vocal_segment)
    features["rms_mean"] = RMSEn.mean()
    features["rms_var"] = RMSEn.var()
    
    # Spectral Centroid
    spec_cent = librosa.feature.spectral_centroid(y=vocal_segment)
    features["spectral_centroid_mean"] = spec_cent.mean()
    features["spectral_centroid_var"] = spec_cent.var()
    
    # Spectral Bandwidth
    spec_band = librosa.feature.spectral_bandwidth(y=vocal_segment, sr=sr)
    features["spectral_bandwidth_mean"] = spec_band.mean()
    features["spectral_bandwidth_var"] = spec_band.var()
    
    # Rolloff
    spec_roll = librosa.feature.spectral_rolloff(y=vocal_segment, sr=sr)
    features["rolloff_mean"] = spec_roll.mean()
    features["rolloff_var"] = spec_roll.var()
    
    # Zero Crossing Rate
    zero_crossing = librosa.feature.zero_crossing_rate(y=vocal_segment)
    features["zero_crossing_rate_mean"] = zero_crossing.mean()
    features["zero_crossing_rate_var"] = zero_crossing.var()
    
    # Harmonics and Percussive
    harmony, perceptr = librosa.effects.hpss(y=vocal_segment)
    features["harmony_mean"] = harmony.mean()
    features["harmony_var"] = harmony.var()
    features["perceptr_mean"] = perceptr.mean()
    features["perceptr_var"] = perceptr.var()
    
    # Tempo
    tempo, _ = librosa.beat.beat_track(y=vocal_segment, sr=sr)
    features["tempo"] = float(tempo)
    
    # MFCCs
    mfcc = librosa.feature.mfcc(y=vocal_segment, sr=sr, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=chroma_hop_length)
    mfcc = mfcc.T
    for x in range(num_mfcc):
        features[f"mfcc{x+1}_mean"] = mfcc[:, x].mean()
        features[f"mfcc{x+1}_var"] = mfcc[:, x].var()
    
    return features

def predict_genre(audio_path):
    # Extract features from the audio file
    features = extract_features(audio_path)
    
    # Prepare the feature vector for prediction
    feature_vector = [features[feat] for feat in features if feat not in ["filename", "label"]]
    feature_vector = np.array(feature_vector).reshape(1, -1)
    
    # Normalize the feature vector using StandardScaler
    feature_vector_normalized = sc.transform(feature_vector)
    
    # Predict the genre using the provided model
    prediction = logistic_regression_model.predict(feature_vector_normalized)
    
    # Map numerical label to genre name
    genre_mapping = {
        0: 'bolero',
        1: 'hiphop',
        2: 'kid',
        3: 'rb',
        4: 'red',
        5: 'rock'
    }
    
    predicted_genre = genre_mapping[prediction[0]]
    
    return predicted_genre


In [3]:
# Example audio file path
audio_path = "4_Ánh Đèn Phố.wav"

# Predict genre
predicted_genre = predict_genre(audio_path)
print(f"The predicted genre is: {predicted_genre}")

The predicted genre is: rock


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18776\2944324134.py:83: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  features["tempo"] = float(tempo)
